#  1. Importações e dados.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
sns.set_theme()

In [3]:
import pandas as pd

# Carregar a base original
df = pd.read_csv('/home/fause/ML-Transportes/PBIC/Etapa3/acidentes_pbic_2020_2025_Final.csv', encoding="utf-8")

df_acidentes = df.drop_duplicates(subset="id").copy()

df.tail()


/tmp/ipykernel_40879/860121245.py:4: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/fause/ML-Transportes/PBIC/Etapa3/acidentes_pbic_2020_2025_Final.csv', encoding="utf-8")


,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_principal,...,mes,dia_mes,hora,gravidade_numerica,total_vitimas,Marca_Principal,Modelo_Grupo,Nome_Modelo,Fabricante,Modelo
1678321,705809.0,1571785.0,2025-05-01,quinta-feira,19:50:00,SE,101.0,"98,6",SAO CRISTOVAO,Sim,...,5,1,19,0,0.0,Não Informado,Não Informado,NO,Não Informado,NO
1678322,705809.0,1571786.0,2025-05-01,quinta-feira,19:50:00,SE,101.0,"98,6",SAO CRISTOVAO,Sim,...,5,1,19,3,1.0,HONDA,CG 125 FAN ES,CG,HONDA,CG
1678323,705809.0,1571785.0,2025-05-01,quinta-feira,19:50:00,SE,101.0,"98,6",SAO CRISTOVAO,Sim,...,5,1,19,0,0.0,Não Informado,Não Informado,NO,Não Informado,NO
1678324,705883.0,1571987.0,2025-06-22,domingo,12:30:00,MG,354.0,764,ITAMONTE,Sim,...,6,22,12,1,1.0,HONDA,CB 650R,CB,HONDA,CB
1678325,705883.0,1571987.0,2025-06-22,domingo,12:30:00,MG,354.0,764,ITAMONTE,Sim,...,6,22,12,1,1.0,HONDA,CB 650R,CB,HONDA,CB


#informações 

In [4]:
# Contar o número de valores únicas na coluna 'marca'
numero_Fabricante_unicas = df['Fabricante'].nunique()
numero_Modelo_unicas = df['Modelo'].nunique()

print(f"O número de marcas de veículos diferentes é: {numero_Fabricante_unicas}")
print(f"O número de modelos diferentes é: {numero_Modelo_unicas}")


O número de marcas de veículos diferentes é: 230
O número de modelos diferentes é: 1500


In [ ]:
# Contar a frequência de cada marca e mostrar as 10 mais frequentes
marcas_mais_frequentes = df['Fabricante'].value_counts().head(10)

print("As 10 Fabricante de veículos que mais aparecem são:")
print(marcas_mais_frequentes)

# Veiculos x Acidentes

In [ ]:
df_veiculos = df[(df['ano_fabricacao_veiculo'] >= 1990) & (df['ano_fabricacao_veiculo'] <= 2025)].copy()


In [ ]:

if {'tipo_veiculo', 'ano_fabricacao_veiculo'}.issubset(df_veiculos.columns):
    fig, axes = plt.subplots(1, 2, figsize=(14,6))

    # Distribuição por tipo de veículo
    df_veiculos['tipo_veiculo'].value_counts().plot(kind='bar', ax=axes[0], color='skyblue')
    axes[0].set_title("Distribuição por Tipo de Veículo")
    axes[0].set_ylabel("Nº de Acidentes")

    # Distribuição por ano de fabricação
    df_veiculos['ano_fabricacao_veiculo'].dropna().astype(int).hist(ax=axes[1], bins=30, color='green')
    axes[1].set_title("Distribuição por Ano de Fabricação")
    axes[1].set_xlabel("Ano de Fabricação")
    axes[1].set_ylabel("Nº de Acidentes")

    plt.tight_layout()
    plt.show()


# Nº de Acidentes por Ano de Fabricação (Top 15 Marcas)

In [ ]:
# Pressupondo que df_veiculos já está carregado e contém 'Nome_Modelo', 'Marca_Principal' e 'ano_fabricacao_veiculo'

# 1. Identificar os top 10 modelos
top_10_modelos = df_veiculos['Marca_Subarvore'].value_counts().nlargest(10).index.tolist()

# 2. Filtrar o DataFrame, limpar e agrupar
df_top_10 = df_veiculos[df_veiculos['Marca_Subarvore'].isin(top_10_modelos)].copy()

# Limpeza e conversão de tipo (igual à sua etapa anterior)
df_top_10['ano_fabricacao_veiculo'] = pd.to_numeric(df_top_10['ano_fabricacao_veiculo'], errors='coerce', downcast='integer')
df_top_10.dropna(subset=['ano_fabricacao_veiculo'], inplace=True)
df_top_10['ano_fabricacao_veiculo'] = df_top_10['ano_fabricacao_veiculo'].astype(int)

# 3. Criar uma coluna de rótulo combinada (Marca - Modelo)
# Primeiro, agrupamos para pegar a marca principal de cada modelo no nosso subset
# Usamos .first() assumindo que Nome_Modelo tem uma única Marca_Principal no subset
# Se houver mais de uma marca para um modelo, isso pegará a primeira que aparecer.
marca_modelo_map = df_top_10.groupby('Marca_Subarvore')['Marca_Arvore'].first().to_dict()

# Mapear a marca de volta para o DataFrame e criar o novo rótulo
df_top_10['Marca_Modelo_Label'] = df_top_10.apply(
    lambda row: f"{marca_modelo_map[row['Marca_Subarvore']]} - {row['Marca_Subarvore']}", axis=1
)

# 4. Agrupamento: Contar o número de acidentes por Ano de Fabricação e pelo novo Rótulo
df_contagem = df_top_10.groupby(['ano_fabricacao_veiculo', 'Marca_Modelo_Label']).size().reset_index(name='Nº de Acidentes')

# 5. Criar o Gráfico de Linhas com Seaborn
plt.figure(figsize=(18, 9))

# Usamos 'hue' para separar as linhas pelo novo rótulo combinado
sns.lineplot(data=df_contagem,
             x='ano_fabricacao_veiculo',
             y='Nº de Acidentes',
             hue='Marca_Modelo_Label',
             palette='tab20',  # Paleta de cores com 20 cores distintas
             linewidth=2)

plt.title('Nº de Acidentes por Ano de Fabricação (Top 10 Modelos com Marca)', fontsize=16)
plt.xlabel('Ano de Fabricação', fontsize=12)
plt.ylabel('Nº de Acidentes', fontsize=12)

# Ajuste do eixo X para mostrar apenas anos inteiros e com rotação
# Garante que os anos no eixo X sejam os únicos e a cada 2 anos, como antes
anos_unicos = df_contagem['ano_fabricacao_veiculo'].unique()
plt.xticks(rotation=45, ha='right', ticks=anos_unicos[::2])

# Ajuste da legenda fora da área de plotagem para não poluir o gráfico
plt.legend(title='Marca - Modelo', bbox_to_anchor=(1.05, 1), loc='upper left', ncol=1)

plt.grid(axis='both', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Distribuição (Top 15 Marca - Modelo)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Remover linhas com CG150
df_veiculos = df_veiculos[~df_veiculos['Marca_Subarvore'].str.contains("CG150", case=False, na=False)]

# --- Pré-processamento e Criação do Rótulo Combinado ---

top_15_modelos = df_veiculos['Marca_Subarvore'].value_counts().nlargest(15).index.tolist()
df_top_15 = df_veiculos[df_veiculos['Marca_Subarvore'].isin(top_15_modelos)].copy()

df_top_15['Marca_Modelo_Label'] = df_top_15.apply(
    lambda row: f"{row['Marca_Arvore']} - {row['Marca_Subarvore']}", axis=1
)

contagem_marca_modelo = df_top_15['Marca_Modelo_Label'].value_counts().nlargest(15)

# --- Geração do Gráfico Único ---
if {'Marca_Subarvore', 'Marca_Arvore'}.issubset(df_veiculos.columns):
    fig, ax = plt.subplots(1, 1, figsize=(12, 7))

    contagem_marca_modelo.plot(
        kind='bar',
        ax=ax,
        color=sns.color_palette("tab10", 15),
        edgecolor='black'
    )
    ax.set_title("Distribuição (Top 15 Marca - Modelo)", fontsize=16)
    ax.set_ylabel("Nº de Acidentes", fontsize=12)
    ax.set_xlabel("Marca - Modelo", fontsize=12)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    ax.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()
else:
    print("O DataFrame 'df_veiculos' precisa das colunas 'Marca_Subarvore' e 'Marca_Arvore'.")


#Top Veículos (Geral) com Mais Acidentes e Tipos de Acidente

In [ ]:

# Definir o valor inválido a ser filtrado (do submodelo)
valor_excluir_exato = 'NO'              # Match exato
substring_excluir = 'CG150'             # Match por substring

# Criar df_limpo aplicando dois filtros:
# 1) remover quem é exatamente 'NO'
# 2) remover qualquer linha que contenha "CG150" (case-insensitive)
df_limpo = df[
    (~df['Marca_Subarvore'].eq(valor_excluir_exato)) &
    (~df['Marca_Subarvore'].str.contains(substring_excluir, case=False, na=False))
].copy()

# --- NOVO PASSO: Criar o mapeamento Marca_Subarvore -> Marca_Arvore ---
try:
    # Cria o mapeamento. Usamos .drop_duplicates() para garantir que
    # o índice (Marca_Subarvore) seja único.
    marca_map = df_limpo[['Marca_Subarvore', 'Marca_Arvore']].drop_duplicates().set_index('Marca_Subarvore')['Marca_Arvore'].to_dict()
except KeyError:
    print("Erro: O DataFrame não possui as colunas 'Marca_Subarvore' e/ou 'Marca_Arvore'. Não será possível gerar o rótulo combinado.")
    marca_map = {}


# --- ANÁLISE GERAL: Top 20 veículos e depois pegar do 1º ao 14º ---

# A contagem agora usa TODOS os tipos de veículos (não apenas 'Automóvel')
top_veiculos = (
    df_limpo.groupby('Marca_Subarvore')['causa_acidente']
    .count()
    .reset_index(name='qtd_acidentes')
    .sort_values('qtd_acidentes', ascending=False)
    .head(20)
)
# Selecionar os 14 primeiros (índices 0 a 13)
top_veiculos = top_veiculos.iloc[0:14] 

n_veiculos = len(top_veiculos)

# Verificação de segurança
if n_veiculos == 0:
    print("A filtragem não resultou em veículos válidos para plotar.")
elif n_veiculos > 0:
    fig, axes = plt.subplots(n_veiculos, 1, figsize=(10, 5*n_veiculos), sharex=False)
    # Título ajustado para refletir a análise geral
    fig.suptitle("Top Veículos (Geral) com Mais Acidentes e Tipos de Acidente", fontsize=16, fontweight='bold')

    if n_veiculos == 1:
        axes = [axes]

    for ax, veiculo in zip(axes, top_veiculos['Marca_Subarvore']):
        
        # Filtrar apenas os acidentes do veículo atual (em TODO o df_limpo)
        df_veiculo = df_limpo[df_limpo['Marca_Subarvore'] == veiculo].copy()
        
        # Top 10 tipos de acidentes
        top_causas = (
            df_veiculo.groupby('causa_acidente')
            .size()
            .reset_index(name='qtd_acidentes')
            .sort_values('qtd_acidentes', ascending=False)
            .head(10)
        )
        
        # Encontrar o ano de fabricação mais frequente para este veículo
        if not df_veiculo.empty and 'ano_fabricacao_veiculo' in df_veiculo.columns:
             ano_mais_frequente = df_veiculo['ano_fabricacao_veiculo'].mode().iloc[0]
        else:
             ano_mais_frequente = "N/A"

        # --- AJUSTE: Criar o Rótulo Combinado para o Título ---
        marca_principal = marca_map.get(veiculo, 'Desconhecida')
        titulo_combinado = f"{marca_principal} - {veiculo}"

        # Plot
        ax.barh(top_causas['causa_acidente'], top_causas['qtd_acidentes'], color='teal')
        
        # Usar o rótulo combinado no título
        ax.set_title(f"Veículo: {titulo_combinado} (Ano mais frequente: {ano_mais_frequente})")
        
        ax.set_xlabel("Quantidade de Acidentes")
        ax.set_ylabel("Causa do Acidente")
        ax.invert_yaxis()

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Não informado 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# 1. Filtra o DataFrame para incluir APENAS os registros onde Marca_Subarvore é 'NO'
# Adiciona padronização (toUpperCase) para capturar 'no', 'No', etc., se existirem.
df_no = df[df['Marca_Subarvore'].astype(str).str.upper() == 'NO'].copy()

# 2. Verifica se há dados suficientes para plotar
if df_no.empty:
    print("O DataFrame filtrado para 'Marca_Subarvore' = 'NO' está vazio. Não é possível gerar os gráficos.")
else:
    # --- GRÁFICO 1: Top 10 Causa_Acidente para 'NO' ---
    top_causas_no = (
        df_no.groupby('causa_acidente')
        .size()
        .reset_index(name='qtd_acidentes')
        .sort_values('qtd_acidentes', ascending=False)
        .head(10)
    )

    # --- GRÁFICO 2: Top 10 Tipo_Acidente para 'NO' ---
    top_tipos_no = (
        df_no.groupby('tipo_acidente')
        .size()
        .reset_index(name='qtd_acidentes')
        .sort_values('qtd_acidentes', ascending=False)
        .head(10)
    )
    
    # 3. Cria a Figura e os Subplots (2 colunas)
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    fig.suptitle("Análise de Acidentes com Veículos de Marca 'Não Informada' (NO)", fontsize=16, fontweight='bold')

    # --- Plotagem do Gráfico 1: Causa do Acidente ---
    if not top_causas_no.empty:
        sns.barplot(
            data=top_causas_no,
            x='qtd_acidentes',
            y='causa_acidente',
            ax=axes[0],
            palette='viridis',
            edgecolor='black'
        )
        axes[0].set_title("Top 10 Causas de Acidente (Marca: NO)", fontsize=14)
        axes[0].set_xlabel("Quantidade de Acidentes", fontsize=12)
        axes[0].set_ylabel("Causa do Acidente", fontsize=12)
        axes[0].grid(axis='x', linestyle='--', alpha=0.7)
    else:
        axes[0].text(0.5, 0.5, "Sem dados de Causa de Acidente para 'NO'.", transform=axes[0].transAxes, ha='center')


    # --- Plotagem do Gráfico 2: Tipo de Acidente ---
    if not top_tipos_no.empty:
        sns.barplot(
            data=top_tipos_no,
            x='qtd_acidentes',
            y='tipo_acidente',
            ax=axes[1],
            palette='magma',
            edgecolor='black'
        )
        axes[1].set_title("Top 10 Tipos de Acidente (Marca: NO)", fontsize=14)
        axes[1].set_xlabel("Quantidade de Acidentes", fontsize=12)
        axes[1].set_ylabel("Tipo de Acidente", fontsize=12)
        axes[1].grid(axis='x', linestyle='--', alpha=0.7)
    else:
        axes[1].text(0.5, 0.5, "Sem dados de Tipo de Acidente para 'NO'.", transform=axes[1].transAxes, ha='center')

    plt.tight_layout(rect=[0, 0, 1, 0.95]) # Ajusta o layout para evitar sobreposição do título
    plt.show()